In [1]:
!pip install streamlit
!pip install pyngrok
!pip install pillow
!pip install easyocr
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 610.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00


In [2]:
%%writefile app07.py

import streamlit as st
import joblib
import pandas as pd
import easyocr
from PIL import Image
from deep_translator import GoogleTranslator

# Load the saved logistic regression model and vectorizer
model = joblib.load('LRmodel.pkl')
vectorizer = joblib.load('LRvectorizer.pkl')

# Initialize OCR reader
reader = easyocr.Reader(['en', 'es', 'fr', 'de'])  # Add other languages if needed

# Define the Streamlit app
def run_app():
    st.title("Mental Health Classifier with Image Text Extraction and Translation")

    # Upload an image in any format (including WebP)
    uploaded_image = st.file_uploader("Upload an Image", type=["jpg", "jpeg", "png", "webp", "bmp", "tiff"])

    if uploaded_image is not None:
        image = Image.open(uploaded_image)
        st.image(image, caption='Uploaded Image', use_column_width=True)

        # Convert the image to text
        extracted_text = reader.readtext(image, detail=0)
        extracted_text = "\n".join(extracted_text)

        st.subheader("Extracted Text")
        st.text(extracted_text)

        # Translate text to English if necessary
        try:
            translated_text = GoogleTranslator(source='auto', target='en').translate(extracted_text)
            st.subheader("Translated Text (to English)")
            st.text(translated_text)
        except Exception as e:
            st.error(f"Translation failed: {e}")
            return

        # Button to make prediction on the extracted text
        if st.button("Classify Mental Health Issue"):
            if translated_text.strip() == "":
                st.write("No valid text found for classification.")
            else:
                # Preprocess and vectorize the translated text
                input_vectorized = vectorizer.transform([translated_text])

                # Make prediction with probabilities
                prediction_proba = model.predict_proba(input_vectorized)

                # Get class labels and probabilities
                issue_labels = model.classes_
                proba_df = pd.DataFrame(prediction_proba, columns=issue_labels)
                proba_df = proba_df.T  # Transpose for readability
                proba_df.columns = ['Probability']

                # Get the mental health issue with the highest probability
                top_issue = proba_df['Probability'].idxmax()
                top_probability = proba_df['Probability'].max()

                # Output the result
                st.write(f"The most likely mental health concern is: {top_issue} with a probability of {top_probability:.2%}")

# Run the app
if __name__ == '__main__':
    run_app()


Writing app07.py


In [ ]:
# Import ngrok
from pyngrok import ngrok

# Set your authtoken
ngrok.set_auth_token("<NGROK_AUTH_TOKEN>") # Replace YOUR_AUTHTOKEN with your actual authtoken

# Kill any existing ngrok processes
ngrok.kill()

# Start Streamlit with nohup
!nohup streamlit run app07.py &

# Create a public URL with ngrok to access the app
public_url = ngrok.connect(addr='8501')
print(f"Public URL: {public_url}")

nohup: appending output to 'nohup.out'
Public URL: NgrokTunnel: "https://1f6e-35-194-168-70.ngrok-free.app" -> "http://localhost:8501"


In [4]:
ngrok.kill()